# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
import csv

csv_file = "raw_api.csv"

df = pd.read_csv(csv_file)

df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,North Bend,43.41,-124.22,59.00,82,1,8.05,US,1603246000
1,Mattawa,46.74,-119.90,57.20,76,40,5.82,US,1603246149
2,Kaitangata,-46.28,169.85,56.82,47,57,2.66,NZ,1603246149
3,Cherskiy,68.75,161.30,23.02,97,97,4.41,RU,1603246149
4,Saskylakh,71.92,114.08,19.78,97,91,8.52,RU,1603246150


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[["Latitude", "Longitude"]]

humidity = df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
husky_cold = df[df["Max Temp"] < 50]

husky_cold.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Cherskiy,68.75,161.30,23.02,97,97,4.41,RU,1603246149
4,Saskylakh,71.92,114.08,19.78,97,91,8.52,RU,1603246150
6,Tasiilaq,65.61,-37.64,32.00,96,100,23.04,GL,1603245980
8,Qaanaaq,77.48,-69.36,20.41,82,37,6.17,GL,1603246150
10,Upernavik,72.79,-56.15,32.74,92,100,2.86,GL,1603246151


In [6]:
bring_on_the_snow = husky_cold[husky_cold["Cloudiness"] > 95]

bring_on_the_snow.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Cherskiy,68.75,161.30,23.02,97,97,4.41,RU,1603246149
6,Tasiilaq,65.61,-37.64,32.00,96,100,23.04,GL,1603245980
10,Upernavik,72.79,-56.15,32.74,92,100,2.86,GL,1603246151
22,Yar-Sale,66.83,70.83,25.30,98,100,3.80,RU,1603246154
39,Yerbogachën,61.28,108.01,21.97,94,100,9.60,RU,1603246157


In [7]:
TrumpFriendly = bring_on_the_snow[bring_on_the_snow["Country"] == "RU"]

TrumpFriendly.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Cherskiy,68.75,161.30,23.02,97,97,4.41,RU,1603246149
22,Yar-Sale,66.83,70.83,25.30,98,100,3.80,RU,1603246154
39,Yerbogachën,61.28,108.01,21.97,94,100,9.60,RU,1603246157
100,Oktyabr'skiy,54.48,53.47,35.56,89,100,8.79,RU,1603246171
128,Sobolevo,54.43,31.90,35.37,87,100,5.59,RU,1603246177


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotelname_df = pd.DataFrame(TrumpFriendly, columns=["City", "Latitude", "Longitude", "Max Temp", "Cloudiness", "Country"])

hotelname_df["Hotel Name"] = ""

hotelname_df

,City,Latitude,Longitude,Max Temp,Cloudiness,Country,Hotel Name
3,Cherskiy,68.75,161.30,23.02,97,RU,
22,Yar-Sale,66.83,70.83,25.30,100,RU,
39,Yerbogachën,61.28,108.01,21.97,100,RU,
100,Oktyabr'skiy,54.48,53.47,35.56,100,RU,
128,Sobolevo,54.43,31.90,35.37,100,RU,
138,Sivaki,52.63,126.75,36.54,98,RU,
145,Nikolskoye,59.70,30.79,32.00,100,RU,
162,Bilibino,68.05,166.44,22.06,98,RU,
167,Turukhansk,65.82,87.98,17.94,100,RU,
189,Ostrovnoy,68.05,39.51,36.77,100,RU,


In [9]:
target_coordinates = f"{hotelname_df['Latitude']},{hotelname_df['Longitude']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {"location": target_coordinates,
          "keyword": target_search,
          "radius": target_radius,
          "type": target_type,
          "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)

In [10]:

hotel_name = []

for index, row in hotelname_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [11]:
hotel_name

['Sberbank',
 'Yalemd',
 nan,
 'Hotel Devon',
 nan,
 'Sivaki',
 nan,
 'Profilaktoriy Baes',
 'ООО "Туруханская пушнина"',
 nan,
 nan,
 'Gostinitsa Kedr',
 'Sberbank Rossii',
 'Uyutnaya Mini-Hotel',
 'Prozhivaniye V Okhotske',
 nan,
 'Gostinitsa',
 'Sberbank',
 'Sberbank Rossii',
 nan,
 'Tarskaya',
 'Sberbank Rossii',
 'Fortuna',
 'Gostinichnyy Kompleks Aylgy',
 'Polar Capital',
 'Gostiny Dvor Evgeniya',
 'Gostinitsa "Udacha"',
 'Gostinitsa "Uyutnyy Dom"']

In [12]:
hotelname_df["Hotel Name"]= hotel_name
hotelname_df

,City,Latitude,Longitude,Max Temp,Cloudiness,Country,Hotel Name
3,Cherskiy,68.75,161.30,23.02,97,RU,Sberbank
22,Yar-Sale,66.83,70.83,25.30,100,RU,Yalemd
39,Yerbogachën,61.28,108.01,21.97,100,RU,NaN
100,Oktyabr'skiy,54.48,53.47,35.56,100,RU,Hotel Devon
128,Sobolevo,54.43,31.90,35.37,100,RU,NaN
138,Sivaki,52.63,126.75,36.54,98,RU,Sivaki
145,Nikolskoye,59.70,30.79,32.00,100,RU,NaN
162,Bilibino,68.05,166.44,22.06,98,RU,Profilaktoriy Baes
167,Turukhansk,65.82,87.98,17.94,100,RU,"ООО ""Туруханская пушнина"""
189,Ostrovnoy,68.05,39.51,36.77,100,RU,NaN


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelname_df.iterrows()]
locations = hotelname_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure

fig

Figure(layout=FigureLayout(height='420px'))